In [1]:
from adabas.api import *
from adabas.datamap import *
a=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)
a.cb.dbid=12              
a.cb.fnr=11 
dicper={}

In [6]:
#campo elementar de um grupo periódico
a.cb.isn=201
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AR'        # elementar short name - FDT
lgn = 'curr_code' # elementar long-name  - DDM
lsn = 3          # length elementar     - DDM
upk = ''         # unpack para colunas packeadas
scl = ''         # scale
Int = ''
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'

a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    ret = eval("""'dicper["{}"] = [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
    #print eval("""'dicper["{}"]= [{}]'.format({})""".format( lgn, '{}, ' * occ, ('row.{ln}{}, '* occ).format(ln=lgn,*range(1,occ+1))))
    print ret

00000201 - occurs 2 times
dicper["address_line"] = ["HEIDELBERGERSTR. 144", "6100 DARMSTADT", ]


In [4]:
#campo elementar de um grupo periódico
a.cb.isn=201
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AS'        # elementar short name - FDT
lgn = 'salary'    # elementar long-name  - DDM
lsn = 9           # length elementar     - DDM
upk = ',U'        # unpack para colunas packeadas
scl = 2           # scale
Int = True
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'
        
a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    ret = eval("""'dicper["{}"] = [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
    #ret = eval("""'dicper["{}"]= [{}]'.format({})""".format( lgn, '{}, ' * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
    print ret

00000201 - occurs 3 times
dicper["salary"] = [246.15, 235.89, 215.38, ]


In [34]:
#campo multiplo de um grupo periódico
a.cb.isn=283
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AT'        # elementar short name - FDT
lgn = 'bonus'     # elementar long-name  - DDM
lsn = 9           # length elementar     - DDM
upk = ',U'        # unpack para colunas packeadas
scl = 0           # scale
Int = True
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'
        
a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    gocc=int(a.rb.value.encode('hex') ) # count occurs group
    for o in range(1,gocc+1):
        a.fb.value='{}{}C,002,B.'.format(esn,o)
        a.get(isn=a.cb.isn)
        if a.rb.value:
            occ=int(a.rb.value.encode('hex') ) # count occurs elementar
            exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
            print '{:08} - occurs {} times'.format(a.cb.isn, occ)
            row.buffer = a.rb
            a.fb.value='{}{}(1-{}),{}{}.'.format(esn, o, occ, lsn, upk)
            a.get(isn=a.cb.isn)
            ret = eval("""'dicper["{}"] = [{}]'.format({})""".format(lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
            print ret
        else:
            break

00000283 - occurs 3 times
dicper["bonus"] = [1846, 2000, 500, ]
00000283 - occurs 1 times
dicper["bonus"] = [1743, ]


In [38]:
#campo elementar múltiplo
a.cb.isn=283
gsn = 'AI'        # elementar short name - FDT
esn = 'AI'        # elementar short name - FDT
lgn = 'address_line'    # elementar long-name  - DDM
lsn = 20           # length elementar     - DDM
upk = ''        # unpack para colunas packeadas

a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    print eval("""'dicper["{}"]= [{}]'.format({})""".format( lgn, '"{}", ' * occ, ('row.{ln}{}, '* occ).format(ln=lgn,*range(1,occ+1))))

00000283 - occurs 2 times
dicper["address_line"]= ["SCHLOEMERWEG 55", "6236 ESCHBORN", ]


In [28]:
a.cb.isn=283
a.fb.value='AQC,2,B.'
a.get(isn=a.cb.isn)
a.rb.value.encode('hex')

'02'

In [29]:
a.fb.value='AT1C,2,B.'
a.get(isn=a.cb.isn)
occ = a.rb.value.encode('hex')
occ

'03'

In [30]:
a.fb.value='AT1(1-{}),9,U.'.format(occ)
a.get(isn=a.cb.isn)
a.rb.value

'000001846000002000000000500000000000000000000'

In [ ]:
a.cb.isn=200
try:
    for c in xrange(10):
        a.fb.value='AQC,002,B.'
        a.readByIsn(getnext=1)
        if a.rb.value:
            aqc=int(a.rb.value.encode('hex') )
            aql=9
            aqu=',9,U'
            exec ("row = Datamap('Rows', {})".format("String('as{}', {l})," * aqc).format(*range(1,aqc+1), l=aql))
            print '{:08} - occurs {} times'.format(a.cb.isn, aqc)
            row.buffer = a.rb
            a.fb.value='AS1-{}{}.'.format(int(aqc), aqu) 
            a.get(isn=a.cb.isn)
            print eval("'{:08} {}'.format({})".format(a.cb.isn, '- {} ' * aqc, ('row.as{}, '* aqc).format(*range(1,aqc+1))))
            
except DataEnd:
    print 'acabou'